# Load libraries and Themes

In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat)
        library(ggplot2)
        library(tidyverse)
        library(SeuratDisk)
       })})

In [ ]:
#Color Palettes

palette.doublets <- c(
    "#197278",#turqu
    "#C1C1C1"#grey
)

palette.2 <- c(
    "#990902",
    "#0303a3"
)

palette.4 <- DiscretePalette(4, palette = "stepped", shuffle = TRUE)

palette.12 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette.15 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)
   
palette.treatment <- c(
    "#AF0000",#ALDO
    "#C1C1C1"#CTRL
)

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

vln_theme_1 <- theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

vln_theme_2 <- theme(
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LA_CTRL_ALDO")

In [ ]:
set.seed(1234)

# Add Subcluster annotations to original file

In [ ]:
obj_rename <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample_rename.h5seurat")

In [ ]:
CM <- LoadH5Seurat("subcluster/Subcluster_CM_full.h5seurat")

In [ ]:
seurat_objects <- list(CM)

In [ ]:
set_cell_type_sub <- function(seurat_objects) {
  for (obj in seurat_objects) {
    obj@meta.data$cell_type_sub <- Idents(obj)
  }
}

set_cell_type_sub(seurat_objects)

In [ ]:
unique(obj_rename@meta.data$cell_type)

In [ ]:
object_wo_subcluster <- subset(x = obj_rename, subset = cell_type %in% c("MΦ", "FB", "TC", "PER", "EC-end", "BC", "EC-cap", "MESO", "EC-lym", "SC"))

In [ ]:
# Create empty cell_type_sub_num column in obj_rename_viableclusters
obj_rename$cell_type_sub <- NA

In [ ]:
# Extract meta.data
original_metadata <- obj_rename@meta.data

wo_subcluster_metadata <- object_wo_subcluster@meta.data
CM_metadata <- CM@meta.data

In [ ]:
# Reset rownames to a column in all data frames
original_metadata$barcode <- rownames(original_metadata)

wo_subcluster_metadata$barcode <- rownames(wo_subcluster_metadata)
CM_metadata$barcode <- rownames(CM_metadata)

In [ ]:
# Subset to barcode and cell_type_sub columns
original_metadata <- original_metadata[c("barcode", "cell_type_sub")]

wo_subcluster_metadata <- wo_subcluster_metadata[c("barcode", "cell_type")]
CM_metadata <- CM_metadata[c("barcode", "cell_type_sub")]

In [ ]:
# Merge df
merged_df <- merge(original_metadata, wo_subcluster_metadata, by = "barcode", all.x = TRUE)
merged_df <- merge(merged_df, CM_metadata, by = "barcode", all.x = TRUE)

In [ ]:
names(merged_df) <- c("barcode", "original", "wo_subcluster", "CM")

In [ ]:
merged_df$merged_column <- coalesce(merged_df$wo_subcluster, merged_df$CM)

In [ ]:
cell_type_sub_df <- merged_df[c("barcode", "merged_column")]

In [ ]:
cell_type_sub_df <- cell_type_sub_df[match(original_metadata$barcode, cell_type_sub_df$barcode), ]

In [ ]:
cell_type_sub_df <- cell_type_sub_df[c("merged_column")]

In [ ]:
cell_type_sub_df <- unlist(cell_type_sub_df, use.names = FALSE)

In [ ]:
obj_rename$cell_type_sub <- cell_type_sub_df

In [ ]:
head(obj_rename@meta.data)

In [ ]:
# Remove cells not correctly assigned due to random subsampling (n=4)

barcodes_to_remove <- c(rownames(obj_rename@meta.data[is.na(obj_rename@meta.data$cell_type_sub), ]))
barcodes_to_remove

# Subset the Seurat object to exclude the specified barcodes by directly using rownames
obj_rename <- obj_rename[, !rownames(obj_rename@meta.data) %in% barcodes_to_remove]

In [ ]:
any(rownames(obj_rename) %in% barcodes_to_remove)

In [ ]:
# Create a contingency table for cell_type_sub and treatment
tab <- table(obj_rename@meta.data$cell_type_sub, obj_rename@meta.data$treatment)

# View the table
tab

In [ ]:
suppressWarnings(print(DimPlot(obj_rename, group.by = "cell_type_sub")))
suppressWarnings(print(DimPlot(obj_rename, group.by = "cell_type")))

In [ ]:
SaveH5Seurat(obj_rename, "seurat_objects/obj_rename_withsubcluster")

In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.1 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/scrna_dm/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] SeuratDisk_0.0.0.9021 lubridate_1.9.4       forcats_1.0.0        
 [4] stringr_1.5.1         dplyr_1.1.4           purrr_1.0.2          
 [7] readr_2.1.5           tidyr_1.3.1           tibble_3.2.1         
[10] tidyverse_2.0.0       ggplot2_3.5.1         Seurat_5.1.0     